# 광주 인공지능 사관학교
- - -
- 작성자 : 2반 한지호
- 작성일 : 20.07.15 수
- 3교시 딥러닝 시간에 진행한 '전복 고리 수 추정' 예제 클론 코딩 실습
- 단층 퍼셉트론의 이해를 위해 예제 코드를 강사님과 함께 보며 구조와 그 이유를 파악하였다.
- A, B Line 까지 오늘 진행했고, 다음 수업시간 진도에 맞춰 추가 작성할 예정

![flowchart](flowchart.png)

### 0.0 파이썬 모듈 불러들이기  
- 이 예제는 단층 퍼셉트론의 기본 구조를 파악하기 위해 라이브러리 사용을 최소화한 예제다.

In [1]:
import numpy as np
import csv

np.random.seed(1234)

### 0.1 하이퍼 파라미터 정의  
- 하이퍼 파라미터 : 고정값 (한 번의 실험 사이클 중간에 절대 변경 불가능한 값)  

In [2]:
RND_MEAN = 0
RND_STD = 0.0030

LEARNING_RATE = 0.001 # 학습률

### A.1 실험용 메인함수

In [3]:
def abalone_exec(epoch_count=10, mb_szie=10, report=1):
    load_abalone_dataset() # 데이터를 불러들이는 함수
    init_model() # 모델 초기화 함수
    train_and_test(epoch_count, mb_szie, report) # 학습 및 테스트 수행 함수

### B.1 데이터 적재함수 정의

In [4]:
def load_abalone_dataset():
    with open('abalone.csv') as f:
        r = csv.reader(f)
        next(r, None) # next()로 첫 행을 None으로 처리 (원하는 데이터만 출력 = 불필요한 정보를 처리)  
        rows = []
        for row in r:
            rows.append(row)
    
    global data, input_count, output_count # 전역변수 생성
    # 데이터의 입출력 벡터 정보 저장. 이후 크기 지정에 활용. 
    input_count, output_count = 10, 1 #원 핫(3) + 속성(7) = 10, 레이블 = 1
    data = np.zeros(len(rows), input_count+output_count) 
    
    # 원-핫 벡터 처리
    # I = 1,0,0 / M = 0,1,0 / F = 0,0,1
    for n, row in enumerate(rows):
        # data[n, 0~3] = 성별 정보 원 핫 벡터 처리 / 이후 데이터(속성)는 그 뒤에 그대로 복사
        if row[0] == 'I': 
            data[n, 0] = 1 
        if row[0] == 'M': 
            data[n, 1] = 1 
        if row[0] == 'F': 
            data[n, 2] = 1 
        data[n, 3:] = row[1:] 

### B.2 파라미터 초기화 함수 정의

In [5]:
def init_model():
    global weight, bias, input_count, output_count # 전역변수 불러오기 및 생성
    weight = np.random.normal(RND_MEAN, RND_STD, [input_count, output_count]) # 가중치 초기화 : 정규분포를 갖는 난수 생성
    bias = np.zeros([output_count]) # 편향 0으로 초기화

### B.3 학습 및 평가 함수 정의
- 1 epoch = 1 batch (= N mini batch)

In [6]:
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size) 
    test_x, test_y = get_test_data() 
    
    for epoch in range(epoch_count): # epoch_count 만큼 epoch 반복 수행
        losses, accs = [], [] # 한 차례의 epoch마다 손실과 정확도 저장
        
        for n in range(step_count): # 학습 데이터 크기에 비례하여 (80%) 미니배치 처리된 횟수 만큼 반복 수행
            train_x, train_y = get_train_data(mb_size, n) # 미니배치 마다의 학습 데이터 분할
            loss, acc = run_train(train_x, train_y) # 학습 수행 및 손실과 정확도 산출
            # 미니배치 처리 이후 손실과 정확도를 누적하여 저장 (이후 이 값들을 평균내면 한차례의 'epoch' 처리) 
            losses.append(loss) 
            accs.append(acc) 
            
        if report > 0 and (epoch+1) % report == 0: # 출력 주기 및 테스트 주기 설정
            acc = run_test(test_X, test_y) # 테스트 데이터로 테스트 진행
            print(f'Epoch {epoch+1}: loss={np.mean(losses):5.3f}, accuracy={np.mean(accs):5.3f}/{acc:5.3f}')
            
        final_acc = run_test(test_x, test_y) # 모든 반복이 종료되었을 때, 한 번 더 최종 결과 출력
        print(f'\nFinal Test: final accuracy = {final_acc:5.3f}') 